<a href="https://colab.research.google.com/github/gnuespy/gnues/blob/main/%EA%B1%B0%EB%A6%AC%EC%82%AC%EB%9E%8C_%EC%B9%B4%EC%9A%B4%ED%8A%B8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 14.4 MB/s eta 0:00:00


In [3]:
import cv2
from ultralytics import YOLO
import numpy as np
from google.colab.patches import cv2_imshow
from collections import deque

# YOLOv8 모델 불러오기
model = YOLO('yolov8n.pt')

# 비디오 캡처 초기화
cap = cv2.VideoCapture("/content/drive/MyDrive/yoloynew/nywalking_exported.mp4")

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/drive/MyDrive/yolonew/output2_with_counts.mp4", fourcc, 30.0,
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 카운팅 변수 초기화
total_in_count = 0  # 누적 입장 수
total_out_count = 0  # 누적 퇴장 수
detected_ids = {}
next_id = 0
exit_buffer = 30  # ID가 삭제되기 전까지 유지되는 프레임 수
id_buffer = deque(maxlen=50)  # 최근 ID 기록용

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 수행
    results = model(frame)

    current_ids = set()  # 현재 프레임에서 감지된 ID를 저장할 집합

    for det in results[0].boxes:
        x1, y1, x2, y2 = map(int, det.xyxy[0])  # 박스 좌표
        conf = det.conf[0].item()  # 신뢰도
        cls = int(det.cls[0].item())  # 클래스

        if cls == 0 and conf > 0.5:  # 클래스 0은 사람

            # 중심 좌표 계산
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # ID 생성 및 관리
            object_id = None
            for id, (prev_x, prev_y, frames) in detected_ids.items():
                if abs(prev_x - center_x) < 50 and abs(prev_y - center_y) < 50:
                    object_id = id
                    detected_ids[id] = (center_x, center_y, exit_buffer)  # 프레임 수 초기화
                    break

            if object_id is None:
                object_id = next_id
                detected_ids[object_id] = (center_x, center_y, exit_buffer)
                next_id += 1
                total_in_count += 1  # 새 ID 감지 시 입장으로 간주

            current_ids.add(object_id)  # 현재 프레임에 감지된 ID 추가

            # 경계 상자와 ID 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'ID: {object_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 이전 프레임에서 존재했지만 현재 프레임에 없는 ID의 프레임 수 감소
    for id in list(detected_ids.keys()):
        if id not in current_ids:
            prev_x, prev_y, frames = detected_ids[id]
            if frames > 0:
                detected_ids[id] = (prev_x, prev_y, frames - 1)
            else:
                del detected_ids[id]
                total_out_count += 1  # ID가 사라진 경우 퇴장으로 간주

    # 누적 카운트 표시
    cv2.putText(frame, f'Total In: {total_in_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Total Out: {total_out_count}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 비디오에 현재 프레임 저장
    out.write(frame)

    # Colab 환경에서 현재 프레임 표시
    #cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 60.0MB/s]



0: 384x640 (no detections), 485.2ms
Speed: 16.0ms preprocess, 485.2ms inference, 16.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 233.5ms
Speed: 2.1ms preprocess, 233.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 231.5ms
Speed: 2.7ms preprocess, 231.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 266.7ms
Speed: 2.0ms preprocess, 266.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 253.6ms
Speed: 2.1ms preprocess, 253.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 238.1ms
Speed: 4.2ms preprocess, 238.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 238.3ms
Speed: 2.1ms preprocess, 238.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 240.7ms
Speed: 2.3ms pre